## IMU_cut_timestamp

In [1]:
from scipy.signal import butter, filtfilt
# 设定滤波器参数

def butter_lowpass(cutoff, fs, order=5):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def apply_lowpass_filter(data, cutoff_freq, fs=1, order=5):
    b, a = butter_lowpass(cutoff_freq, fs, order=order)
    smoothed_data = filtfilt(b, a, data)
    return smoothed_data


In [5]:

import pandas as pd
import os
import time

def imu_preprocess(folder_path):
    # 获取指定文件夹下的所有文件和文件夹
    contents = os.listdir(folder_path)
    timestamp_path='C:\\Users\\lyy\\Desktop\\Data\\Raw\\Timestamp'
    cutoff_frequency = 7.5 # 截止频率
    sampling_frequency = 100  # 采样频率
    filter_order = 4  # 滤波器阶数
    for item in contents[:]:
        item_path = os.path.join(folder_path, item)
        
        #浏览对应item的timestamp文件夹
        item_timestamp= os.path.join(timestamp_path,item)
        print('folder：' + item_timestamp)
        
        # 检查是否为文件夹
        if os.path.isdir(item_path):
            print('folder：' + item_path)
        
        #遍历item_kinect下的子文件first、second;遍历item_timestamp下的子文件first、second
        item_contents = os.listdir(item_path)#first和second文件名
        
        for t in item_contents[:]:
            item_path_time = os.path.join(item_path, t) #第item次子文件地址
            item_timestamp_time= os.path.join(item_timestamp, t) #第item次时间戳
            print('folder：'+item_path_time)
            print('folder：'+item_timestamp_time)
            
            
            #浏览item_timestamp的对应次数时间戳
            timestamp=pd.read_csv(item_timestamp_time+'.csv')
                  
            file_contents = os.listdir(item_path_time)   
            
            #IMU前两个样本文件记录的格式与后面样本的格式有轻微差别          
            
            if item=='01' or item=='02':
                
                for file in file_contents[:4]:
                    if file[:-4] == 'task_normal':
                        row = 0
                    if file[:-4] == 'task_cup':
                        row = 1
                    if file[:-4] == 'task_text':
                        row = 2
                    if file[:-4] == 'task_web':
                        row = 3
                    #导入文件
                    file_path = os.path.join(item_path_time, file)
                    print('file：'+file_path)

                    df_1= pd.read_csv(file_path,index_col=False)
                    

                    
                    #时间戳转化为数字格式
                    for j in range(6):
                        time_array_imu= time.strptime(timestamp.iloc[row,j+1][:-7], "%Y-%m-%d-%H:%M:%S")
                        timestamp_imu = time.mktime(time_array_imu)
                        timestamp_imu= timestamp_imu+float(timestamp.iloc[row,j+1][-7:-3])
                        timestamp.iloc[row,j+1]=timestamp_imu
                    
                    #划分时间戳节点
                    start=timestamp.iloc[row,1]
                    start_kinect=timestamp.iloc[row,2]
                    start_turn=timestamp.iloc[row,3]
                    end_turn=timestamp.iloc[row,4]
                    end_kinect=timestamp.iloc[row,5]
                    end=timestamp.iloc[row,6]
                    
                    imu_new = df_1.copy()
                    for i in range(0,len(imu_new)):
                        timestamp2 = imu_new.iloc[i,0]
                        if timestamp2 >= start:
                            imu_new.drop(imu_new.head(i).index,inplace=True)
                            break
                    imu_new.reset_index(drop=True,inplace=True)

                    for i in range(0,len(imu_new)):
                        timestamp2 = imu_new.iloc[i,0]
                        if timestamp2 > end :
                            imu_new.drop(imu_new.tail(len(imu_new)-i).index,inplace=True)
                            break
                            
                    imu_new.iloc[:,2:11].apply(lambda x: apply_lowpass_filter(x, cutoff_frequency, fs=sampling_frequency, order=filter_order))

                        
                    #分段截取
                    #截取go阶段:start 到 start_turn
                    imu_go = imu_new.copy()
                    for i in range(0,len(imu_go)):
                        timestamp2 = imu_go.iloc[i,0]
                        if timestamp2 >= start:
                            imu_go.drop(imu_go.head(i).index,inplace=True)
                            break
                    imu_go.reset_index(drop=True,inplace=True)

                    for i in range(0,len(imu_go)):
                        timestamp2 = imu_go.iloc[i,0]
                        if timestamp2 > start_turn :
                            imu_go.drop(imu_go.tail(len(imu_go)-i).index,inplace=True)
                            break
                    imu_go['时间'] = imu_go['时间'].astype(str)
                    
                   
                    
                    
                    if len(imu_go)<2:
                        print("错误")     
                    #角度矫正
                    first_row_value_x = imu_go.loc[0,'角度X(°)']
                    first_row_value_y = imu_go.loc[0,'角度Y(°)']
                    first_row_value_z = imu_go.loc[0,'角度Z(°)']
                    # 将指定列的每一行减去第一行的值
            
                    imu_go.loc[:,'角度X(°)'] = imu_go['角度X(°)'] - first_row_value_x
                    imu_go.loc[:,'角度Y(°)'] = imu_go['角度Y(°)'] - first_row_value_y
                    imu_go.loc[:,'角度Z(°)'] = imu_go['角度Z(°)'] - first_row_value_z
                        
            
                    imu_go.to_csv("C:\\Users\\lyy\\Desktop\\Data\\Processed\\IMU_5m\\"+t+'\\go\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False)
                    
                    #截取go_kinect阶段:start_kinect 到 start_turn
                    imu_go_kinect = imu_new.copy()
                    for i in range(0,len(imu_go_kinect)):
                        timestamp2 = imu_go_kinect.iloc[i,0]
                        if timestamp2 >= start_kinect:
                            imu_go_kinect.drop(imu_go_kinect.head(i).index,inplace=True)
                            break
                    imu_go_kinect.reset_index(drop=True,inplace=True)
                    for i in range(0,len(imu_go_kinect)):
                        timestamp2 = imu_go_kinect.iloc[i,0]
                        if timestamp2 > start_turn :
                            imu_go_kinect.drop(imu_go_kinect.tail(len(imu_go_kinect)-i).index,inplace=True)
                            break
                    imu_go_kinect['时间'] = imu_go_kinect['时间'].astype(str)
                    
                    #角度矫正
                    first_row_value_x = imu_go_kinect.loc[0,'角度X(°)']
                    first_row_value_y = imu_go_kinect.loc[0,'角度Y(°)']
                    first_row_value_z = imu_go_kinect.loc[0,'角度Z(°)']
                    # 将指定列的每一行减去第一行的值
            
                    imu_go_kinect.loc[:,'角度X(°)'] = imu_go_kinect['角度X(°)'] - first_row_value_x
                    imu_go_kinect.loc[:,'角度Y(°)'] = imu_go_kinect['角度Y(°)'] - first_row_value_y
                    imu_go_kinect.loc[:,'角度Z(°)'] = imu_go_kinect['角度Z(°)'] - first_row_value_z
                    
                    imu_go_kinect.to_csv("C:\\Users\\lyy\\Desktop\\Data\\Processed\\IMU_2m\\"+t+'\\go\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False)          
                    
                   
                    #截取turn阶段:start_turn 到 end_turn
                    imu_turn = imu_new.copy()
                    for i in range(0,len(imu_turn)):
                        timestamp2 = imu_turn.iloc[i,0]
                        if timestamp2 >= start_turn:
                            imu_turn.drop(imu_turn.head(i).index,inplace=True)
                            break
                    imu_turn.reset_index(drop=True,inplace=True)
                    for i in range(0,len(imu_turn)):
                        timestamp2 = imu_turn.iloc[i,0]
                        if timestamp2 > end_turn :
                            imu_turn.drop(imu_turn.tail(len(imu_turn)-i).index,inplace=True)
                            break
                    imu_turn['时间'] = imu_turn['时间'].astype(str)
                    if len(imu_turn)<2:
                        print("错误")                            
                   
                    #角度矫正
                    first_row_value_x = imu_turn.loc[0,'角度X(°)']
                    first_row_value_y = imu_turn.loc[0,'角度Y(°)']
                    first_row_value_z = imu_turn.loc[0,'角度Z(°)']
                    # 将指定列的每一行减去第一行的值
            
                    imu_turn.loc[:,'角度X(°)'] = imu_turn['角度X(°)'] - first_row_value_x
                    imu_turn.loc[:,'角度Y(°)'] = imu_turn['角度Y(°)'] - first_row_value_y
                    imu_turn.loc[:,'角度Z(°)'] = imu_turn['角度Z(°)'] - first_row_value_z
                    
                    imu_turn.to_csv("C:\\Users\\lyy\\Desktop\\Data\\Processed\\IMU_5m\\"+t+'\\turn\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False) 
                    imu_turn.to_csv("C:\\Users\\lyy\\Desktop\\Data\\Processed\\IMU_2m\\"+t+'\\turn\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False)
                    
                    #截取back阶段:end_turn 到 end
                    imu_back = imu_new.copy()
                    for i in range(0,len(imu_back)):
                        timestamp2 = imu_back.iloc[i,0]
                        if timestamp2 >= end_turn:
                            imu_back.drop(imu_back.head(i).index,inplace=True)
                            break
                    imu_back.reset_index(drop=True,inplace=True)
                    for i in range(0,len(imu_back)):
                        timestamp2 = imu_back.iloc[i,0]
                        if timestamp2 > end :
                            imu_back.drop(imu_back.tail(len(imu_back)-i).index,inplace=True)
                            break
                    imu_back['时间'] = imu_back['时间'].astype(str)
                   
                    #角度矫正
                    first_row_value_x = imu_back.loc[0,'角度X(°)']
                    first_row_value_y = imu_back.loc[0,'角度Y(°)']
                    first_row_value_z = imu_back.loc[0,'角度Z(°)']
                    # 将指定列的每一行减去第一行的值
            
                    imu_back.loc[:,'角度X(°)'] = imu_back['角度X(°)'] - first_row_value_x
                    imu_back.loc[:,'角度Y(°)'] = imu_back['角度Y(°)'] - first_row_value_y
                    imu_back.loc[:,'角度Z(°)'] = imu_back['角度Z(°)'] - first_row_value_z
                    imu_back.to_csv("C:\\Users\\lyy\\Desktop\\Data\\Processed\\IMU_5m\\"+t+'\\back\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False) 
                       
                    
                    #截取back_kinect阶段:end_turn 到 end_kinect
                    imu_back_kinect = imu_new.copy()
                    for i in range(0,len(imu_back_kinect)):
                        timestamp2 = imu_back_kinect.iloc[i,0]
                        if timestamp2 >= end_turn:
                            imu_back_kinect.drop(imu_back_kinect.head(i).index,inplace=True)
                            break
                    imu_back_kinect.reset_index(drop=True,inplace=True)
                    for i in range(0,len(imu_back_kinect)):
                        timestamp2 = imu_back_kinect.iloc[i,0]
                        if timestamp2 > end_kinect :
                            imu_back_kinect.drop(imu_back_kinect.tail(len(imu_back_kinect)-i).index,inplace=True)
                            break
                    imu_back_kinect['时间'] = imu_back_kinect['时间'].astype(str)
                    
                    #角度矫正
                    first_row_value_x = imu_back_kinect.loc[0,'角度X(°)']
                    first_row_value_y = imu_back_kinect.loc[0,'角度Y(°)']
                    first_row_value_z = imu_back_kinect.loc[0,'角度Z(°)']
                    # 将指定列的每一行减去第一行的值
            
                    imu_back_kinect.loc[:,'角度X(°)'] = imu_back_kinect['角度X(°)'] - first_row_value_x
                    imu_back_kinect.loc[:,'角度Y(°)'] = imu_back_kinect['角度Y(°)'] - first_row_value_y
                    imu_back_kinect.loc[:,'角度Z(°)'] = imu_back_kinect['角度Z(°)'] - first_row_value_z
                    
                    imu_back_kinect.to_csv("C:\\Users\\lyy\\Desktop\\Data\\Processed\\IMU_2m\\"+t+'\\back\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False)                 
            else:
                
                for file in file_contents[:4]:
                    if file[:-4] == 'task_normal':
                        row = 0
                    if file[:-4] == 'task_cup':
                        row = 1
                    if file[:-4] == 'task_text':
                        row = 2
                    if file[:-4] == 'task_web':
                        row = 3
                    #导入文件
                    file_path = os.path.join(item_path_time, file)
                    print('file：'+file_path)
                    df_2=pd.read_csv(file_path,index_col=False)#读取文件
                    
      
                    #时间戳转化为数字格式
                    for j in range(6):
                        time_array_imu= time.strptime(timestamp.iloc[row,j+1][:-7], "%Y-%m-%d-%H:%M:%S")
                        timestamp_imu = time.mktime(time_array_imu)
                        timestamp_imu= timestamp_imu+float(timestamp.iloc[row,j+1][-7:-3])
                        timestamp.iloc[row,j+1]=timestamp_imu
                            
                    #划分时间戳节点
                    start=timestamp.iloc[row,1]
                    start_kinect=timestamp.iloc[row,2]
                    start_turn=timestamp.iloc[row,3]
                    end_turn=timestamp.iloc[row,4]
                    end_kinect=timestamp.iloc[row,5]
                    end=timestamp.iloc[row,6]
                    
                    imu_new = df_2.copy()
                    for i in range(0,len(imu_new)):
                        timestamp2 = imu_new.iloc[i,0]
                        if timestamp2 >= start:
                            imu_new.drop(imu_new.head(i).index,inplace=True)
                            break
                    imu_new.reset_index(drop=True,inplace=True)

                    for i in range(0,len(imu_new)):
                        timestamp2 = imu_new.iloc[i,0]
                        if timestamp2 > end :
                            imu_new.drop(imu_new.tail(len(imu_new)-i).index,inplace=True)
                            break
                    imu_new.iloc[:,3:12].apply(lambda x: apply_lowpass_filter(x, cutoff_frequency, fs=sampling_frequency, order=filter_order))        
             
              
                    #截取go阶段:start 到 start_turn
                    imu_go = imu_new.copy()
                    for i in range(0,len(imu_go)):
                        timestamp2 = imu_go.iloc[i,0]
                        if timestamp2 >= start:
                            imu_go.drop(imu_go.head(i).index,inplace=True)
                            break
                    imu_go.reset_index(drop=True,inplace=True)
                    for i in range(0,len(imu_go)):
                        timestamp2 = imu_go.iloc[i,0]
                        if timestamp2 > start_turn :
                            imu_go.drop(imu_go.tail(len(imu_go)-i).index,inplace=True)
                            break
                    imu_go['时间'] = imu_go['时间'].astype(str)
                    
                    if len(imu_go)<2:
                        print("错误")   
                    #角度矫正
                    first_row_value_x = imu_go.loc[0,'角度X(°)']
                    first_row_value_y = imu_go.loc[0,'角度Y(°)']
                    first_row_value_z = imu_go.loc[0,'角度Z(°)']
                    # 将指定列的每一行减去第一行的值
            
                    imu_go.loc[:,'角度X(°)'] = imu_go['角度X(°)'] - first_row_value_x
                    imu_go.loc[:,'角度Y(°)'] = imu_go['角度Y(°)'] - first_row_value_y
                    imu_go.loc[:,'角度Z(°)'] = imu_go['角度Z(°)'] - first_row_value_z    
                    
                    imu_go.to_csv("C:\\Users\\lyy\\Desktop\\Data\\Processed\\IMU_5m\\"+t+'\\go\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False)
                   
                    #截取go_kinect阶段:start_kinect 到 start_turn
                    imu_go_kinect = imu_new.copy()
                    for i in range(0,len(imu_go_kinect)):
                        timestamp2 = imu_go_kinect.iloc[i,0]
                        if timestamp2 >= start_kinect:
                            imu_go_kinect.drop(imu_go_kinect.head(i).index,inplace=True)
                            break
                    imu_go_kinect.reset_index(drop=True,inplace=True)
                    for i in range(0,len(imu_go_kinect)):
                        timestamp2 = imu_go_kinect.iloc[i,0]
                        if timestamp2 > start_turn :
                            imu_go_kinect.drop(imu_go_kinect.tail(len(imu_go_kinect)-i).index,inplace=True)
                            break
                    imu_go_kinect['时间'] = imu_go_kinect['时间'].astype(str)
                    
                    #角度矫正
                    first_row_value_x = imu_go_kinect.loc[0,'角度X(°)']
                    first_row_value_y = imu_go_kinect.loc[0,'角度Y(°)']
                    first_row_value_z = imu_go_kinect.loc[0,'角度Z(°)']
                    # 将指定列的每一行减去第一行的值
            
                    imu_go_kinect.loc[:,'角度X(°)'] = imu_go_kinect['角度X(°)'] - first_row_value_x
                    imu_go_kinect.loc[:,'角度Y(°)'] = imu_go_kinect['角度Y(°)'] - first_row_value_y
                    imu_go_kinect.loc[:,'角度Z(°)'] = imu_go_kinect['角度Z(°)'] - first_row_value_z 
                    
                    imu_go_kinect.to_csv("C:\\Users\\lyy\\Desktop\\Data\\Processed\\IMU_2m\\"+t+'\\go\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False)         
                    
                    
                    #截取turn阶段:start_turn 到 end_turn
                    imu_turn = imu_new.copy()
                    for i in range(0,len(imu_turn)):
                        timestamp2 = imu_turn.iloc[i,0]
                        if timestamp2 >= start_turn:
                            imu_turn.drop(imu_turn.head(i).index,inplace=True)
                            break
                    imu_turn.reset_index(drop=True,inplace=True)
                    for i in range(0,len(imu_turn)):
                        timestamp2 = imu_turn.iloc[i,0]
                        if timestamp2 > end_turn :
                            imu_turn.drop(imu_turn.tail(len(imu_turn)-i).index,inplace=True)
                            break
                    imu_turn['时间'] = imu_turn['时间'].astype(str)
                    
                    if len(imu_turn)<2:
                        print("错误")      
                    #角度矫正
                    first_row_value_x = imu_turn.loc[0,'角度X(°)']
                    first_row_value_y = imu_turn.loc[0,'角度Y(°)']
                    first_row_value_z = imu_turn.loc[0,'角度Z(°)']
                    # 将指定列的每一行减去第一行的值
            
                    imu_turn.loc[:,'角度X(°)'] = imu_turn['角度X(°)'] - first_row_value_x
                    imu_turn.loc[:,'角度Y(°)'] = imu_turn['角度Y(°)'] - first_row_value_y
                    imu_turn.loc[:,'角度Z(°)'] = imu_turn['角度Z(°)'] - first_row_value_z     
                    
                    imu_turn.to_csv("C:\\Users\\lyy\\Desktop\\Data\\Processed\\IMU_5m\\"+t+'\\turn\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False) 
                    imu_turn.to_csv("C:\\Users\\lyy\\Desktop\\Data\\Processed\\IMU_2m\\"+t+'\\turn\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False) 
                    
                    #截取back阶段:end_turn 到 end
                    imu_back = imu_new.copy()
                    for i in range(0,len(imu_back)):
                        timestamp2 = imu_back.iloc[i,0]
                        if timestamp2 >= end_turn:
                            imu_back.drop(imu_back.head(i).index,inplace=True)
                            break
                    imu_back.reset_index(drop=True,inplace=True)
                    for i in range(0,len(imu_back)):
                        timestamp2 = imu_back.iloc[i,0]
                        if timestamp2 > end :
                            imu_back.drop(imu_back.tail(len(imu_back)-i).index,inplace=True)
                            break
                    imu_back['时间'] = imu_back['时间'].astype(str)
                    
                    #角度矫正
                    first_row_value_x = imu_back.loc[0,'角度X(°)']
                    first_row_value_y = imu_back.loc[0,'角度Y(°)']
                    first_row_value_z = imu_back.loc[0,'角度Z(°)']
                    # 将指定列的每一行减去第一行的值
            
                    imu_back.loc[:,'角度X(°)'] = imu_back['角度X(°)'] - first_row_value_x
                    imu_back.loc[:,'角度Y(°)'] = imu_back['角度Y(°)'] - first_row_value_y
                    imu_back.loc[:,'角度Z(°)'] = imu_back['角度Z(°)'] - first_row_value_z  
                    
                    imu_back.to_csv("C:\\Users\\lyy\\Desktop\\Data\\Processed\\IMU_5m\\"+t+'\\back\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False)    
                    
                    #截取back_kinect阶段:end_turn 到 end_kinect
                    imu_back_kinect = imu_new.copy()
                    for i in range(0,len(imu_back_kinect)):
                        timestamp2 = imu_back_kinect.iloc[i,0]
                        if timestamp2 >= end_turn:
                            imu_back_kinect.drop(imu_back_kinect.head(i).index,inplace=True)
                            break
                    imu_back_kinect.reset_index(drop=True,inplace=True)
                    for i in range(0,len(imu_back_kinect)):
                        timestamp2 = imu_back_kinect.iloc[i,0]
                        if timestamp2 > end_kinect :
                            imu_back_kinect.drop(imu_back_kinect.tail(len(imu_back_kinect)-i).index,inplace=True)
                            break
                    imu_back_kinect['时间'] = imu_back_kinect['时间'].astype(str)
                    #imu_back_kinect.iloc[:,2:11].apply(lambda x: apply_lowpass_filter(x, cutoff_frequency, fs=sampling_frequency, order=filter_order))
                    #角度矫正
                    first_row_value_x = imu_back_kinect.loc[0,'角度X(°)']
                    first_row_value_y = imu_back_kinect.loc[0,'角度Y(°)']
                    first_row_value_z = imu_back_kinect.loc[0,'角度Z(°)']
                    # 将指定列的每一行减去第一行的值
            
                    imu_back_kinect.loc[:,'角度X(°)'] = imu_back_kinect['角度X(°)'] - first_row_value_x
                    imu_back_kinect.loc[:,'角度Y(°)'] = imu_back_kinect['角度Y(°)'] - first_row_value_y
                    imu_back_kinect.loc[:,'角度Z(°)'] = imu_back_kinect['角度Z(°)'] - first_row_value_z  
                    
                    imu_back_kinect.to_csv("C:\\Users\\lyy\\Desktop\\Data\\Processed\\IMU_2m\\"+t+'\\back\\'+file[:-4]+'\\'+item+'_'+file[:-4]+'.csv',index=False)                 

                        

                
                
        
       
            

In [6]:
imu_preprocess("C:\\Users\\lyy\\Desktop\\Data\\Raw\\IMU")

文件夹：C:\Users\lyy\Desktop\Data\Raw\Timestamp\sample01
文件夹：C:\Users\lyy\Desktop\Data\Raw\IMU\sample01
文件夹：C:\Users\lyy\Desktop\Data\Raw\IMU\sample01\time01
文件夹：C:\Users\lyy\Desktop\Data\Raw\Timestamp\sample01\time01
文件：C:\Users\lyy\Desktop\Data\Raw\IMU\sample01\time01\task_cup.csv
文件：C:\Users\lyy\Desktop\Data\Raw\IMU\sample01\time01\task_normal.csv
文件：C:\Users\lyy\Desktop\Data\Raw\IMU\sample01\time01\task_text.csv
文件：C:\Users\lyy\Desktop\Data\Raw\IMU\sample01\time01\task_web.csv
文件夹：C:\Users\lyy\Desktop\Data\Raw\IMU\sample01\time02
文件夹：C:\Users\lyy\Desktop\Data\Raw\Timestamp\sample01\time02
文件：C:\Users\lyy\Desktop\Data\Raw\IMU\sample01\time02\task_cup.csv
文件：C:\Users\lyy\Desktop\Data\Raw\IMU\sample01\time02\task_normal.csv
文件：C:\Users\lyy\Desktop\Data\Raw\IMU\sample01\time02\task_text.csv
文件：C:\Users\lyy\Desktop\Data\Raw\IMU\sample01\time02\task_web.csv
文件夹：C:\Users\lyy\Desktop\Data\Raw\Timestamp\sample02
文件夹：C:\Users\lyy\Desktop\Data\Raw\IMU\sample02
文件夹：C:\Users\lyy\Desktop\Data\Raw\IM